# Read trade info

In [ ]:
import json
import csv

# see https://docs.rmrk.app/syncing/#consolidation -> "RMRK 2 consolidated dump" for files
with open('rmrk20211118.json') as json_file: # rmrk20211017.json # QmQ8sz7ejw1DemKgQ21xBpzeKiCtYdJUuKQEAaZGq1jjhX.json
    data = json.load(json_file)

In [ ]:
collection_id = 'e0b9bdcc456a36497a-KANBIRD'
birds_number = 0
birds = list()
for nft_key in data['nfts'].keys():
    if collection_id in nft_key:
        birds.append(data['nfts'][nft_key])
        birds_number = birds_number + 1
print(birds_number)

In [ ]:
trade_events = list()
for bird_index in range(len(birds)):
    bird_id = int(birds[bird_index]["id"][-8:])
    for operation in birds[bird_index]["changes"]:
        if (operation["opType"] == "BUY") and (operation["field"] == "forsale"):
            block = operation["block"]
            price = float(operation["old"]) / 1000000000000 / 0.95 # [KSM] with 5% of protocol fee (5% may change over time; hardcoded for now)
            trade_events.append([bird_index, bird_id, price, block])
print("The number of deals: ", len(trade_events))
print("The number of unique birds with deals: ", len(set([x[1] for x in trade_events])))

In [ ]:
# !pip install substrate-interface

In [ ]:
from substrateinterface import SubstrateInterface

substrate = SubstrateInterface(
    url="wss://kusama-rpc.polkadot.io/",
    ss58_format=2,
    type_registry_preset='kusama'
)

In [ ]:
# try Kusama RPC
block_hash = "0x350687795aa02bb23fa84154bf6ed3d87b9af444cc30d873526d3e11068378da"
block_height = 9104828

# Retrieve extrinsics in block
# result = substrate.get_block(block_hash=block_hash)
result = substrate.get_block(block_number=block_height)

for extrinsic in result['extrinsics']:
    for param in extrinsic["call"]['call_args']:
        print(type(param))
        if not isinstance(param, str) and (param['name'] == 'now'):
            print(param['value'])
        elif isinstance(param, str) and (param == 'now'):
            print(extrinsic["call"]['call_args']['now'])


'''for extrinsic in result['extrinsics']:

    if 'address' in extrinsic:
        signed_by_address = extrinsic['address'].value
    else:
        signed_by_address = None

    print('\nPallet: {}\nCall: {}\nSigned by: {}'.format(
        extrinsic["call"]["call_module"].name,
        extrinsic["call"]["call_function"].name,
        signed_by_address
    ))

    # Loop through call params
    for param in extrinsic["call"]['call_args']:

        if param['type'] == 'Balance':
            param['value'] = '{} {}'.format(param['value'] / 10 ** substrate.token_decimals, substrate.token_symbol)

        print("Param '{}': {}".format(param['name'], param['value']))'''

In [ ]:
substrate = SubstrateInterface(
    url="wss://kusama-rpc.polkadot.io/",
    ss58_format=2,
    type_registry_preset='kusama'
)

for trade_event_index in range(len(trade_events)):
    if (trade_event_index % 100 == 0):
        print(trade_event_index)
    trade_event = trade_events[trade_event_index]
    block_height = int(trade_event[3])
    success = 0
    while not success:
        try:
            result = substrate.get_block(block_number=block_height)
            success = 1
        except:
            block_height= block_height - 1
            print('Problem: ', trade_event_index)
    trade_value = 0
    for extrinsic in result['extrinsics']:
            for param in extrinsic["call"]['call_args']:
                if not isinstance(param, str) and (param['name'] == 'now'):
                    trade_value = param['value']
                elif isinstance(param, str) and (param == 'now'):
                    trade_value = extrinsic["call"]['call_args']['now']
    assert (trade_value != 0), [trade_event, result]
    trade_event.append(trade_value)
    trade_events[trade_event_index] = trade_event
print("done")

In [ ]:
column_names = ['bird_index', 'bird_id', 'price', 'block', 'time']
with open('trade_events.csv', 'w', newline='') as trade_events_file:
    trade_events_writer = csv.writer(trade_events_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    trade_events_writer.writerow(column_names)
    for row_index in range(len(trade_events)):
        trade_events_writer.writerow(trade_events[row_index])

In [ ]:
trade_events = list()
with open('trade_events.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            trade_events.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')